<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-25 13:45:26
-------------------
qualified stocks: 93
with latest results: 30
still star stocks: 18
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.34 C
-------------------
Today PnL: 1.29 L (0.97%)
Current PnL: -15.68 L (-10.93%)
CY Booked + Current PnL: -9.19 L (-6.4%)
-------------------
Total profit:  3.28 L
Total loss:  -18.96 L
-------------------
Total Booked + Current PnL: 17.79 L (15.26%)
Total Booked PnL: 33.47 L (28.72%)
Curr Year Booked PnL: 6.49 L (4.86%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 72.56 L (54.3%)
Deployed:  1.17 C
Current:  1.34 C
CAGR/XIRR %: 9.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1291.83,1526.00,7.63,X-LC,51.61,160088.0,19279.0,6243.0,1.23,13.69,3.90,18.13,18.0,3.09,1.20,26.41,XY25,NTT,REFINERIES
84,UJJIVANSFB,52.77,53.00,52.86,M-SC,86.02,132840.0,-9639.0,10255.0,0.96,-6.77,7.72,0.44,242.0,-0.94,0.99,56.79,OX40N,NTT,BANKS
89,VIPIND,488.80,489.00,-971.00,H-SC,97.85,82963.0,-12353.0,12395.0,-2.50,-12.96,14.94,0.04,153.0,-1.00,0.62,63.63,OX40N,NTT,MISC
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98796.0,20434.0,13594.0,0.14,26.08,13.76,43.43,79.0,1.50,0.74,32.78,XY24,NTT,MISC
35,HINDALCO,651.95,761.55,-10.07,H-LC,49.46,107872.0,3560.0,13980.0,0.72,3.41,12.96,16.81,27.0,0.25,0.81,20.01,X5K,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-33.09,M-MC,55.91,178869.0,-939.0,113349.0,0.89,-0.52,63.37,62.52,170.0,-0.01,1.34,33.39,XR,NTT,BANKS
50,KPIGREEN,497.21,732.16,NaN,NaN,75.27,125647.0,350.0,58853.0,1.07,0.28,46.84,47.25,137.0,0.01,0.94,57.19,MH,ATH,POWER
34,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,33.33,150892.0,-21.0,60478.0,0.44,-0.01,40.08,40.06,102.0,-0.00,1.13,23.42,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-37.21,H-SC,58.06,100043.0,-461.0,31103.0,-0.52,-0.46,31.09,30.49,97.0,-0.01,0.75,31.80,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SAMMAANCAP,170.35,326.00,-172.08,M-SC,94.62,87126.0,-15084.0,108472.0,16.47,-14.76,124.50,91.37,203.0,-0.14,0.65,41.43,XY25,NTT,FINANCE
25,DEN,52.79,75.00,120.21,L-SC,87.10,62080.0,-22384.0,57921.0,9.57,-26.50,93.30,42.07,257.0,-0.39,0.46,31.84,AR,NTT,ENTERTAINMENT
60,RAJESHEXPO,517.65,518.00,1817.93,M-SC,43.01,52335.0,-84842.0,84934.0,6.69,-61.85,162.29,0.07,232.0,-1.00,0.39,30.16,OX40N,NTT,JEWELLERY
31,HATHWAY,19.74,31.40,1023.19,L-SC,70.97,64440.0,-14520.0,61160.0,6.27,-18.39,94.91,59.07,262.0,-0.24,0.48,29.29,XR,NTT,ENTERTAINMENT
4,ALKYLAMINE,2347.98,4546.37,7.41,X-SC,100.00,99975.0,-988.0,95516.0,5.79,-0.98,95.54,93.63,65.0,-0.01,0.75,48.91,SR,ATH,CHEMICALS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DIXON,14367.00,18940.15,-14.64,H-LC,10.75,112848.0,-2088.0,38673.0,-2.76,-1.82,34.27,31.83,99.0,-0.05,0.84,29.34,X40N,ATH,IT
89,VIPIND,488.80,489.00,-971.00,H-SC,97.85,82963.0,-12353.0,12395.0,-2.50,-12.96,14.94,0.04,153.0,-1.00,0.62,63.63,OX40N,NTT,MISC
65,SAIL,130.64,228.00,94.40,M-MC,44.09,221449.0,-3513.0,171158.0,-1.64,-1.56,77.29,74.53,186.0,-0.02,1.66,28.57,XY24,BTT,STEEL
85,UNIONBANK,123.87,163.00,-6.44,M-LC,59.14,164501.0,23661.0,20826.0,-1.64,16.80,12.66,31.59,90.0,1.14,1.23,42.89,XY24,NTT,BANKS
59,QUESS,711.66,986.00,NaN,NaN,4.30,65218.0,-90636.0,150712.0,-1.34,-58.15,231.09,38.55,190.0,-0.60,0.49,0.00,XY24,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,52.86,M-SC,86.02,132840.0,-9639.0,10255.0,0.96,-6.77,7.72,0.44,242.0,-0.94,0.99,56.79,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-971.00,H-SC,97.85,82963.0,-12353.0,12395.0,-2.50,-12.96,14.94,0.04,153.0,-1.00,0.62,63.63,OX40N,NTT,MISC
78,TATAELXSI,7332.28,7332.0,-16.48,X-MC,32.26,87689.0,-14963.0,14960.0,-0.04,-14.58,17.06,-0.00,58.0,-1.00,0.66,33.30,OX40N,NTT,IT
42,INDIGOPNTS,1407.73,1408.0,122.10,H-SC,89.25,145700.0,-28859.0,28892.0,1.05,-16.53,19.83,0.02,128.0,-1.00,1.09,26.34,OX40N,NTT,PAINTS
76,SYMPHONY,1306.42,1306.0,-33.36,M-SC,2.15,141860.0,-29281.0,29223.0,0.83,-17.11,20.60,-0.03,188.0,-1.00,1.06,1.96,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-17.76,X-LC,27.96,268094.0,-27840.0,121741.0,1.38,-9.41,45.41,31.73,1.0,-0.23,2.01,7.31,X40,BTT,IT
44,INFY,1461.46,2275.00,-9.55,X-LC,62.37,289801.0,28200.0,117427.0,2.36,10.78,40.52,55.67,2.0,0.24,2.17,17.54,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.73,X-LC,20.43,255304.0,-15043.0,49682.0,0.72,-5.56,19.46,12.81,6.0,-0.30,1.91,6.77,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,14.68,X-LC,60.22,263120.0,33887.0,33390.0,1.83,14.78,12.69,29.35,7.0,1.01,1.97,25.02,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-11.33,X-LC,29.03,281572.0,16146.0,40772.0,1.72,6.08,14.48,21.44,9.0,0.40,2.11,12.66,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-1.49,M-LC,38.71,98796.0,20434.0,13594.0,0.14,26.08,13.76,43.43,79.0,1.5,0.74,32.78,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,5.78,H-SC,76.34,118908.0,16904.0,28122.0,0.47,16.57,23.65,44.14,124.0,0.60,0.89,35.50,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98796.0,20434.0,13594.0,0.14,26.08,13.76,43.43,79.0,1.50,0.74,32.78,XY24,NTT,MISC
92,WIPRO,243.46,420.00,-2.45,M-LC,79.57,167307.0,16362.0,93090.0,1.84,10.84,55.64,72.51,69.0,0.18,1.25,15.25,XR,NTT,IT
85,UNIONBANK,123.87,163.00,-6.44,M-LC,59.14,164501.0,23661.0,20826.0,-1.64,16.80,12.66,31.59,90.0,1.14,1.23,42.89,XY24,NTT,BANKS
39,ICICIGI,1839.64,2260.25,-9.50,H-MC,84.95,149036.0,12903.0,18227.0,1.37,9.48,12.23,22.86,35.0,0.71,1.12,23.29,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-30.25,M-SC,82.80,203477.0,31856.0,130205.0,0.70,18.56,63.99,94.43,196.0,0.24,1.52,50.18,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-2.45,M-LC,79.57,167307.0,16362.0,93090.0,1.84,10.84,55.64,72.51,69.0,0.18,1.25,15.25,XR,NTT,IT
41,INDIAMART,2327.09,4850.92,-46.43,H-SC,92.47,139443.0,16107.0,117662.0,5.46,13.06,84.38,108.45,121.0,0.14,1.04,38.33,AR,ATH,MISC
50,KPIGREEN,497.21,732.16,NaN,NaN,75.27,125647.0,350.0,58853.0,1.07,0.28,46.84,47.25,137.0,0.01,0.94,57.19,MH,ATH,POWER
34,HEROMOTOCO,4311.81,6039.03,-0.95,H-MC,33.33,150892.0,-21.0,60478.0,0.44,-0.01,40.08,40.06,102.0,-0.00,1.13,23.42,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,96.77,72576.0,-40973.0,81024.0,1.26,-36.08,111.64,35.27,261.0,-0.51,0.54,77.63,XR,NTT,HOTELS
66,SAMMAANCAP,170.35,326.00,-172.08,M-SC,94.62,87126.0,-15084.0,108472.0,16.47,-14.76,124.50,91.37,203.0,-0.14,0.65,41.43,XY25,NTT,FINANCE
54,MASFIN,326.60,402.39,-19.15,H-SC,72.04,92055.0,-5925.0,28666.0,2.08,-6.05,31.14,23.21,141.0,-0.21,0.69,33.51,XR,ATH,FINANCE
3,ADANIPORTS,1103.69,1583.00,-1.49,M-LC,38.71,98796.0,20434.0,13594.0,0.14,26.08,13.76,43.43,79.0,1.50,0.74,32.78,XY24,NTT,MISC
4,ALKYLAMINE,2347.98,4546.37,7.41,X-SC,100.00,99975.0,-988.0,95516.0,5.79,-0.98,95.54,93.63,65.0,-0.01,0.75,48.91,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,7.41,X-SC,100.00,99975.0,-988.0,95516.0,5.79,-0.98,95.54,93.63,65.0,-0.01,0.75,48.91,SR,ATH,CHEMICALS
7,ASIANTILES,75.41,137.00,7211.11,M-SC,98.92,80238.0,-13572.0,90188.0,-0.80,-14.47,112.40,81.67,240.0,-0.15,0.60,55.38,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-971.00,H-SC,97.85,82963.0,-12353.0,12395.0,-2.50,-12.96,14.94,0.04,153.0,-1.00,0.62,63.63,OX40N,NTT,MISC
22,COFFEEDAY,59.14,80.00,-51.71,L-SC,96.77,72576.0,-40973.0,81024.0,1.26,-36.08,111.64,35.27,261.0,-0.51,0.54,77.63,XR,NTT,HOTELS
48,JPPOWER,18.73,26.20,-33.07,L-SC,95.70,135708.0,-6134.0,62711.0,3.23,-4.32,46.21,39.88,259.0,-0.10,1.02,37.53,XY24,NTT,POWER


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.16
1,25,42.35
2,50,71.06


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.79
LC    37.05
MC    21.51
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.08
XY25     13.40
X40      12.43
X40N     10.60
XR       10.39
OX40N     8.33
AR        7.89
MH        1.69
X5K       1.47
X200      1.47
SR        1.24
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.49
X-LC    15.36
M-SC    14.00
H-LC    12.36
M-LC     8.24
H-MC     8.12
X-MC     7.49
M-MC     5.54
X-SC     2.80
L-SC     2.50
L-LC     1.09
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.33,-2.17,36.35
IT,12.03,-10.54,65.43
FINANCE,7.95,-13.56,54.67
BANKS,7.92,-7.74,51.70
MISC,6.69,-20.97,68.16
PAINTS,5.61,-16.29,39.21
ELECTRICAL,5.04,-4.26,49.40
HEALTHCARE,4.50,-7.28,38.61
AUTO,4.40,-14.99,65.56


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2779654.0,25
XR,1071905.0,13
AR,1014746.0,9
X40,659741.0,9
X40N,513803.0,10
XY25,495616.0,9
OX40N,316899.0,11
SR,179094.0,2
MH,89956.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1893778.0,19
M-SC,1457267.0,18
X-LC,612791.0,9
M-MC,604631.0,5
H-LC,600448.0,11
H-MC,436917.0,6
X-MC,416883.0,6
M-LC,342057.0,6
L-SC,262816.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,873665.0,6
M-SC,XY24,597714.0,5
H-SC,AR,517060.0,3
M-MC,XY24,399108.0,3
M-SC,XR,333888.0,4
X-LC,X40,315533.0,3
H-LC,X40N,239977.0,5
M-LC,XY24,233170.0,4
H-MC,XY24,231797.0,2


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
